# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import datetime as dt
import json
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=3ll3nhsu")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
song1 = pd.read_json(song_files[0], lines=True)
song1

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [5]:
songs_df = pd.DataFrame()

for i in range(len(song_files)):
    filepath = song_files[i]
    songs_df = songs_df.append(pd.read_json(filepath, lines=True))
    
songs_df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
songs = songs_df[['song_id', 'title', 'artist_id', 'year', 'duration']]
    
for i in range(len(songs)):
    song_data = songs.values[i].tolist()
    cur.execute(song_table_insert, song_data)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [7]:
artists = songs_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
    
for i in range(len(artists)):
    artist_data = artists.values[i].tolist()
    cur.execute(artist_table_insert, artist_data)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [8]:
log_files = get_files('data/log_data')
log1 = pd.read_json(log_files[0], lines=True)
log1.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [9]:
log_df = pd.DataFrame()

for i in range(len(log_files)):
    filepath = log_files[i]
    log_df = log_df.append(pd.read_json(filepath, lines=True))
    
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [10]:
log_df = log_df.loc[log_df.page=="NextSong"]
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [22]:
t = log_df.ts.values
t = t.reshape(-1,1)
t.shape

(6820, 1)

In [23]:
time_data = []

# 'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'
for i in range(len(t)):
    try:
        timestamp=()
        start_time = dt.datetime.fromtimestamp(t[i][0] / 1000)
        start_time2 = dt.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S.%f')
        timestamp = (start_time, start_time2.strftime('%H'), start_time2.strftime('%d'), \
                     start_time2.strftime('%U'), start_time2.strftime('%m'), \
                    start_time2.strftime('%Y'), start_time2.strftime('%u'))
        time_data.append(timestamp)
    except KeyError:
        pass

time_data = tuple(time_data)
time_data

((datetime.datetime(2018, 11, 10, 18, 33, 56, 796000),
  '18',
  '10',
  '44',
  '11',
  '2018',
  '6'),
 (datetime.datetime(2018, 11, 10, 18, 36, 10, 796000),
  '18',
  '10',
  '44',
  '11',
  '2018',
  '6'),
 (datetime.datetime(2018, 11, 10, 18, 40, 34, 796000),
  '18',
  '10',
  '44',
  '11',
  '2018',
  '6'),
 (datetime.datetime(2018, 11, 10, 20, 36, 13, 796000),
  '20',
  '10',
  '44',
  '11',
  '2018',
  '6'),
 (datetime.datetime(2018, 11, 10, 20, 36, 46, 796000),
  '20',
  '10',
  '44',
  '11',
  '2018',
  '6'),
 (datetime.datetime(2018, 11, 11, 1, 56, 28, 796000),
  '01',
  '11',
  '45',
  '11',
  '2018',
  '7'),
 (datetime.datetime(2018, 11, 11, 2, 0, 47, 796000),
  '02',
  '11',
  '45',
  '11',
  '2018',
  '7'),
 (datetime.datetime(2018, 11, 11, 2, 4, 7, 796000),
  '02',
  '11',
  '45',
  '11',
  '2018',
  '7'),
 (datetime.datetime(2018, 11, 11, 2, 8, 28, 796000),
  '02',
  '11',
  '45',
  '11',
  '2018',
  '7'),
 (datetime.datetime(2018, 11, 11, 2, 12, 0, 796000),
  '02',
  

In [13]:
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [14]:
time_df = pd.DataFrame(columns=column_labels, data=time_data)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-10 18:33:56.796,18,10,44,11,2018,6
1,2018-11-10 18:36:10.796,18,10,44,11,2018,6
2,2018-11-10 18:40:34.796,18,10,44,11,2018,6
3,2018-11-10 20:36:13.796,20,10,44,11,2018,6
4,2018-11-10 20:36:46.796,20,10,44,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [16]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.dropna(inplace=True)
user_df

/Users/christoph/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
...,...,...,...,...,...
353,83,Stefany,White,F,free
354,83,Stefany,White,F,free
355,26,Ryan,Smith,M,free
356,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [17]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [18]:
for index, row in log_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, [row.song, row.artist, row.length])
    results = cur.fetchone()
        
    if results:
        songId, artistId = results
    else:
        songId, artistId = None, None

    # insert songplay record
    timestamp = dt.datetime.fromtimestamp(row.ts / 1000)
    songplay_data = (timestamp, row.userId, row.level, songId, artistId, \
                     row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
        

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [19]:
cur.close()
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.